# Problem Set 1

## Question 4

Continuing with the model (4), we numerically investigate the performance of various estimators designed for static panel data models, namely, $\hat\beta_{n,\mathrm{POLS}}$, $\hat\beta_{n,\mathrm{FE}}$, $\hat\beta_{n,\mathrm{FD}}$, and the first differenced estimator using $Y_{i,t-2}$ as instrument (i.e., the IV estimator of $\beta_0$ for the first-differenced equation using $Y_{i,t-2}$ as instrument). First, for each triple $(n,T,\beta_0)$ to be specified shortly, obtain the sample as follows.

1. Simulate $\alpha_i\overset{\mathrm{iid}}{\sim} N(0,1)$ and $\epsilon_i\overset{\mathrm{iid}}{\sim} N(0,1)$.

2. Generate $Y_{i0}$ according to $Y_{i0}=0.5\alpha_i + \epsilon_i$.

3. Generate $Y_{it}$ according to (4) with $V_{it}\overset{\mathrm{iid}}{\sim} N(0,1)$.

You may then compute the aforementioned estimators with the generated sample. Repeat the above steps $1000$ times, let $\hat\beta^{(r)}$ be the estimate based on the $i$th replication (for a particular estimator), and then the finite sample bias, standard error and root mean squared error are computed as follows:

\begin{align}
    \mathrm{Bias}(\hat\beta)&=\frac{1}{1000}\sum_{r=1}^{1000}\hat\beta^{(r)} - \beta_0~,\\
    \mathrm{SE}(\hat\beta)&=\{\frac{1}{1000}\sum_{r=1}^{1000}(\hat\beta^{(r)}-\frac{1}{1000}\sum_{r=1}^{1000}\hat\beta^{(r)})^2 \}^{1/2}~,\\
    \mathrm{RMSE} (\hat\beta) & = \{[\mathrm{Bias}(\hat\beta)]^2 + [\mathrm{SE}(\hat\beta)]^2\}^{1/2}~.
\end{align}

1. Let $n=100$ and $T=6$. Graph the bias, se and rmse of each estimator as functions of $\beta_0\in\{0,0.25,0.5,0.75,1\}$.

In [1]:
import os
import numpy as np
from scipy.linalg import toeplitz
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import linearmodels.panel as lm

def DGP(n,T,b0 = 0.5):
    a = np.random.normal(0,1,n)
    e = np.random.normal(0,1,n)
    
    Y0 = 0.5*a + e  # n by 1
    Ymatrix = np.zeros([n, T])  # n by T then stack
    Xmatrix = np.zeros([n, T])
    for t in range(T):
        V = np.random.normal(0,1,n)
        if t == 0:
            Y1 = b0*Y0 + a + V
            Ymatrix[:,t] = Y1
            Xmatrix[:,t] = Y0
        else:
            Y0 = Ymatrix[:,t-1]
            Y1 = b0*Y0 + a + V
            Ymatrix[:,t] = Y1
            Xmatrix[:,t] = Y0
    
    Y = Ymatrix.reshape(-1)
    X = Xmatrix.reshape(-1)
    i = np.kron(np.array([i+1 for i in range(n)]).reshape(n,1), np.ones(T).reshape(T,1)).reshape(-1)
    t = np.kron(np.ones(n).reshape(n,1), np.array([i+1 for i in range(T)]).reshape(T,1)).reshape(-1)
    df = pd.DataFrame({'i':i,'t':t,'Y':Y,'X':X})
    
    return df

def test_DGP(n,T,b0 = 0.5):
    """
    X is exogenous
    """
    a = np.random.normal(0,1,n)
    e = np.random.normal(0,1,n)
    
    Y0 = 0.5*a + e  # n by 1
    Ymatrix = np.zeros([n, T])  # n by T -> stack using reshape.
    Xmatrix = np.zeros([n, T])
    for t in range(T):
        V = np.random.normal(0,1,n)
        X = np.random.normal(0,1,n)
        Y = b0 * X + a + V        
        Ymatrix[:,t] = Y
        Xmatrix[:,t] = X
    Y = Ymatrix.reshape(-1)
    X = Xmatrix.reshape(-1)
    i = np.kron(np.array([i+1 for i in range(n)]).reshape(n,1), np.ones(T).reshape(T,1)).reshape(-1)
    t = np.kron(np.ones(n).reshape(n,1), np.array([i+1 for i in range(T)]).reshape(T,1)).reshape(-1)
    df = pd.DataFrame({'i':i,'t':t,'Y':Y,'X':X})
    
    return df

def my_OLS(Y,X):
    beta = np.linalg.inv(X.T@X) @ (X.T@Y)
    
    return beta[0]

def my_FE(Y,X,T):
    nT, k = X.shape
    n = int(nT/T)
    Qi = np.eye(T) - (np.ones(T).reshape(T,1) @ np.ones(T).reshape(1,T))/T
    Q = np.kron(np.eye(n), Qi)
    
    beta = np.linalg.inv(X.T @ Q @ X) @ (X.T @ Q @ Y)
    
    return beta[0]

def my_FD(Y,X,T):
    nT,k = X.shape
    n = int(nT/T)
    
    toep = toeplitz([-1,*np.zeros(T-2)],[0,1,*np.zeros(T-2)])
    dY = np.kron(np.eye(n),toep) @ Y
    dX = np.kron(np.eye(n),toep) @ X
    
    beta = my_OLS(dY,dX)
    return beta[0]

def my_FD_IV(Y,X,T, method="2SLS"):
    nT,k = X.shape
    n = int(nT/T)
    
    toep = toeplitz([-1,*np.zeros(T-2)],[0,1,*np.zeros(T-2)])[1:,:]
    dY = np.kron(np.eye(n),toep) @ Y
    dX = np.kron(np.eye(n),toep) @ X
    
    sel = np.eye(T)[:-2,:]
    Z = np.kron(np.eye(n),sel) @ Y
    
#     Y_nbyT = Y.reshape(n, T)  # [Y_{t=1}, Y_{t=2}, ..., Y_{t=T}]
#     X_nbyT = X.reshape(n, T)
#     Y_nbyT_lag = np.column_stack([np.zeros(n), Y_nbyT[:,:-1]])  # [0,t=2-1,t=3-2,...]
#     X_nbyT_lag = np.column_stack([np.zeros(n), X_nbyT[:,:-1]])
    
#     dY = (Y_nbyT-Y_nbyT_lag)[:,2:].reshape([n*(T-2),1])  # vec[Y_{t=3-2}, Y_{t=4-3}, ...]
#     dX = (X_nbyT-X_nbyT_lag)[:,2:].reshape([n*(T-2),k])  # vec[Y_{t=2-1}, Y_{t=3-2}, ...]
#     iv = Y_nbyT[:,:-2].reshape([n*(T-2),1])  # vec[Y_{t=1}, Y_{t=2}, ..., Y_{t=T-2}]
    
    # 2SLS
    if method == "2SLS":
        dXhat = Z @ (np.linalg.inv(Z.T@Z) @ (Z.T@dX))
        beta = my_OLS(dY,dXhat)
    if method == "IV":
        nom = dX.T@Z @ np.linalg.inv(Z.T@Z) @ Z.T@dY
        denom = dX.T@Z @ np.linalg.inv(Z.T@Z) @ Z.T@dX
        beta = np.linalg.inv(denom) @ nom
        beta = beta[0]
    if method == "IV_just":
        beta = np.linalg.inv(Z.T@dX) @ Z.T@dY

    return beta

def simulation_Q3(n,T,M=1000,test=False):
    res = []
    for b0 in [0, 0.25, 0.5, 0.75, 1]:
        res_POLS = np.zeros(M)
        res_FE = np.zeros(M)
        res_FD = np.zeros(M)
        res_FD_IV = np.zeros(M)
    
        for m in range(M):
            if test is True:
                df = test_DGP(n=n,T=T,b0=b0)
                Y = np.array(df.Y).reshape(n*T,1)
                X = np.array(df.X).reshape(n*T,1)
            else:
                df = DGP(n=n,T=T,b0=b0)
                Y = np.array(df.Y).reshape(n*T,1)
                X = np.array(df.X).reshape(n*T,1)
            res_POLS[m] = my_OLS(Y,X)
            res_FE[m] = my_FE(Y,X,T=T)
            res_FD[m] = my_FD(Y,X,T=T)
            res_FD_IV[m] = my_FD_IV(Y,X,T=T,method="2SLS")
    
        res.append({"POLS":res_POLS, "FE":res_FE,"FD":res_FD,"FD_IV":res_FD_IV})
    return res

def figure_Q3(res, n, T, save=False):
    b0 = np.array([0,0.25,0.5,0.75,1.])
    result_000 = res[0]
    
    result_000_POLS = result_000["POLS"]
    
    result_025 = res[1]
    result_050 = res[2]
    result_075 = res[3]
    result_100 = res[4]
          
    bias_POLS = [res[i]["POLS"].mean() - b0[i] for i in range(len(res))]
    bias_FE = [res[i]["FE"].mean() - b0[i] for i in range(len(res))]
    bias_FD = [res[i]["FD"].mean() - b0[i] for i in range(len(res))]
    bias_FD_IV = [res[i]["FD_IV"].mean() - b0[i] for i in range(len(res))]

    std_POLS = [res[i]["POLS"].std() for i in range(len(res))]
    std_FE = [res[i]["FE"].std() for i in range(len(res))]
    std_FD = [res[i]["FD"].std() for i in range(len(res))]
    std_FD_IV = [res[i]["FD_IV"].std() for i in range(len(res))]
    
    rmse_POLS = [np.sqrt(bias_POLS[i]**2 + std_POLS[i]**2) for i in range(len(res))]
    rmse_FE = [np.sqrt(bias_FE[i]**2 + std_FE[i]**2) for i in range(len(res))]
    rmse_FD = [np.sqrt(bias_FD[i]**2 + std_FD[i]**2) for i in range(len(res))]
    rmse_FD_IV = [np.sqrt(bias_FD_IV[i]**2 + std_FD_IV[i]**2) for i in range(len(res))]
    
    fig, ax = plt.subplots(1,3,figsize = (21,4))
    
    ax[0].plot(b0, bias_POLS)
    ax[0].plot(b0, bias_FE)
    ax[0].plot(b0, bias_FD)
    ax[0].plot(b0, bias_FD_IV)
    ax[0].axhline(0, color='black')
    ax[0].legend(["POLS", "FE", "FD", "FD_IV"])
    ax[0].set_xlabel('b0')
    ax[0].set_title("Bias of simulation (n={},T={})".format(n,T))

    ax[1].plot(b0, std_POLS)
    ax[1].plot(b0, std_FE)
    ax[1].plot(b0, std_FD)
    ax[1].plot(b0, std_FD_IV)
    ax[1].legend(["POLS", "FE", "FD", "FD_IV"])
    ax[1].set_xlabel('b0')
    ax[1].set_title("SE of simulation (n={},T={})".format(n,T))

    ax[2].plot(b0, rmse_POLS)
    ax[2].plot(b0, rmse_FE)
    ax[2].plot(b0, rmse_FD)
    ax[2].plot(b0, rmse_FD_IV)
    ax[2].legend(["POLS", "FE", "FD", "FD_IV"])
    ax[2].set_xlabel('b0')
    ax[2].set_title("RMSE of simulation (n={},T={})".format(n,T))
    
    fig.savefig('fig_n{}_T{}.jpg'.format(n,T), dpi=80)

    return fig

In [ ]:
results_n100_T6 = simulation_Q3(n=100,T=6,M=1000)
fig_n100_T6 = figure_Q3(results_n100_T6, n=100,T=6, save=True)

2 . Repeat part (1) with $(n,T)=(100,3)$ and $(n,T)=(100,9)$.

In [ ]:
n,T = 100,3
results_n100_T3 = simulation_Q3(n=n,T=T,M=1000)
fig_n100_T3 = figure_Q3(results_n100_T3, n=n,T=T, save=True)

n,T = 100,9
results_n100_T9 = simulation_Q3(n=n,T=T,M=1000)
fig_n100_T9 = figure_Q3(results_n100_T9, n=n,T=T, save=True)

In [ ]:
from pydynpd import regression
_data = test_DGP(n=100,T=6,b0=0.5)
command_str='Y L1.Y  | gmm(Y, 2:.) | nolevel'
mydpd = regression.abond(command_str, _data, ['i', 't'])

mydpd.models[0].regression_table.coefficient[0]

In [ ]:
_data = DGP(100,6,0)
_Y = np.array(_data.Y).reshape(600,1)
_X = np.array(_data.X).reshape(600,1)

np.column_stack([_Y,_X])

my_FD_IV(_Y,_X,T=6)

## Question 5

Do Exercise 17.17(a) in Hansen (2022). You will need the formula (17.93) in Hansen (2022) (see the pdf file in Canvas) for the clustered standard errors.

In [1]:
import os
import numpy as np
from scipy.linalg import toeplitz
import pandas as pd
from pydynpd import regression
file = os.path.join(os.getcwd(),"..","Econometrics Data","Invest1993","Invest1993.txt")
df = pd.read_csv(file, sep='\t')
df.head(3)
#df.year.unique().max() - df.year.unique().min() + 1

,cusip,year,pstar,z0,pricef,divf,rnda,adva,fyr,ardsic,...,debta,sales,netcap,earnsh,nyseamex,h0,h1,vala,oneper,sharef
0,32,1970,4.45946,31.21624,10.000,0.0,0.0,0.00000,12,11,...,0.04644,45.33499,22.64583,0.17,1,53.51015,19.18859,1.17240,-0.48167,2655.0
1,32,1971,5.18880,31.13281,5.750,0.0,0.0,0.00000,12,11,...,0.66158,47.03299,19.28410,-0.65,1,54.77370,16.40725,0.79165,-0.19062,2655.0
2,32,1972,5.66540,28.32700,5.125,0.0,0.0,0.00543,12,11,...,0.57597,34.36200,14.90893,0.72,1,52.25176,12.75835,0.91267,-0.694,2655.0


$$
D_{it} = \alpha D_{i,t-1} + u_i + \epsilon_{it}
$$

(a) Estimate the model using Arellano-Bond twostep GMM with clustered se.

In [3]:
command_str='debta L1.debta  | gmm(debta, 2:.) | nolevel'
mydpd = regression.abond(command_str, df, ['cusip', 'year'])

 Dynamic panel-data estimation, two-step difference GMM
 Group variable: cusip                             Number of obs = 23642    
 Time variable: year                               Min obs per group: 3     
 Number of instruments = 465                       Max obs per group: 30    
 Number of groups = 1962                           Avg obs per group: 12.05 
+----------+-----------+---------------------+------------+-----------+-----+
|  debta   |   coef.   | Corrected Std. Err. |     z      |   P>|z|   |     |
+----------+-----------+---------------------+------------+-----------+-----+
| L1.debta | 0.5754460 |      0.0565248      | 10.1804141 | 0.0000000 | *** |
+----------+-----------+---------------------+------------+-----------+-----+
Hansen test of overid. restrictions: chi(464) = 510.883 Prob > Chi2 = 0.065
Arellano-Bond test for AR(1) in first differences: z = -4.21 Pr > z =0.000
Arellano-Bond test for AR(2) in first differences: z = 1.21 Pr > z =0.226



In [56]:
import janitor
from scipy.linalg import block_diag

df = pd.read_csv(file, sep='\t')

#def my_Arellano_Bond(data):
exog = ['debta']
endog = ['L1.debta']
iv = ['L2.debta']
ID = 'cusip'
T = 'year'

data = df[['cusip','year','debta']].copy()
data = data.sort_values([ID,T])
Tmax = np.max(data.groupby('cusip').year.count())

# generate lagged variables
for name in [*endog,*iv]:
    if name.split('.')[0] == 'L1':  # assume T increases by 1
        data[name] = data[[ID, name.split('.')[-1]]].groupby(ID).shift(1)
    if name.split('.')[0] == 'L2':
        data[name] = data[[ID, name.split('.')[-1]]].groupby(ID).shift(2)
        
# generate differenced varialbes
for name in [*exog, *endog]:
    if name.split('.')[0] == 'L1':
        namesplit = name.split('.')
        data['D.'+name] = data[name] - data['L2.'+namesplit[-1]]
    else:
        data['D.'+name] = data[name] - data['L1.'+name]

# # force balanced panel
# data_dropna = data.dropna()
# data_balanced = data_dropna.complete(ID,T)
# data_balanced = data_balanced.sort_values([ID,T]).fillna(0).reset_index(drop=True)

# generate Y and X
data_balanced = data.dropna()
_Y = np.array(data_balanced[exog]).reshape(-1,1)
_X = data_balanced[endog]
_X = np.array(_X).reshape(-1,_X.shape[1])
_T = len(data_balanced[T].unique())
_n = len(data_balanced[ID].unique())

# build instrument
data = data_balanced
Xlist, Ylist, Zlist = [], [], []  ## CREATE LIST AND DO OPTIMIZATION
IDlist = data[ID].unique()
for i in IDlist:
    data_i = data[data[ID]==i]
    Z_it_list = []
    for t,yr in enumerate(data_i[T].unique()):
        if t <= 2:
            pass
        else:
            Z_it = data_i[data_i[T]<=yr]
#            Z_it = Z_it[[ID,'D.L1.debta']]
            Z_it = Z_it[ID]
            Z_it = np.array(Z_it.dropna()).reshape(1,-1)
            Z_it_list.append(Z_it)
    Z_i = block_diag(*Z_it_list)
    Zlist.append(Z_i)
#Z = np.vstack(Zlist)

In [57]:
data_i = data[data[ID]==32]
data_i[data_i[T]<=1976][[ID]]


,cusip
2,32
3,32
4,32
5,32
6,32


In [55]:
np.array(data_i[data_i[T]<=1976][[ID,'D.L1.debta']]).reshape(1,-1)

array([[ 3.2000e+01,  6.1514e-01,  3.2000e+01, -8.5610e-02,  3.2000e+01,
        -5.8260e-02,  3.2000e+01,  2.9900e-02,  3.2000e+01, -2.0329e-01]])

In [37]:
data

,cusip,year,debta,L1.debta,L2.debta,D.debta,D.L1.debta
2,32,1972,0.57597,0.66158,0.04644,-0.08561,0.61514
3,32,1973,0.51771,0.57597,0.66158,-0.05826,-0.08561
4,32,1974,0.54761,0.51771,0.57597,0.02990,-0.05826
5,32,1975,0.34432,0.54761,0.51771,-0.20329,0.02990
6,32,1976,0.55432,0.34432,0.54761,0.21000,-0.20329
...,...,...,...,...,...,...,...
27561,989845,1987,-0.00855,0.74615,0.39355,-0.75470,0.35260
27562,989845,1988,0.00000,-0.00855,0.74615,0.00855,-0.75470
27563,989845,1989,0.00000,0.00000,-0.00855,0.00000,0.00855
27564,989845,1990,0.17137,0.00000,0.00000,0.17137,0.00000


In [31]:
Zlist[0].shape

(30, 465)

In [33]:
_T

30

In [32]:
Z.shape

# Homoskedasticity
D_i = toeplitz([-1,*np.zeros(_T-2)],[0,1,*np.zeros(_T-2)])
H = D_i @ D_i.T
H.shape
#Omega1 = np.sum()
#Omega1 = Z.T @ np.kron(np.eye(_n),H) @ Z
#Omega1.shape

(29, 29)

In [ ]:
tt = 3
toeplitz([-1,*np.zeros(3-2)],[0,1,*np.zeros(3-2)])

In [ ]:
denom = (_X.T @ Z @ np.linalg.inv(Omega) @ Z.T @ _X)
nom = (_X.T @ Z @ np.linalg.inv(Omega) @ Z.T @ _Y)
gmm = np.linalg.inv(denom)@nom

In [ ]:
Zlist[0].shape

In [ ]:
#data[['cusip','year','debta','L1.debta','L2.debta', 'D.debta', 'D.L1.debta']].head(20)

In [ ]:
df=data_dropna
df_balanced = df.complete('cusip','year')
df_balanced.sort_values(['cusip','year']).head(50)

In [ ]:
df.complete(('ID', 'City', 'State'), 'Date')


In [ ]:
block_diag(A,B)

In [ ]:
data[['cusip','year','debta','L1.debta','L2.debta', 'D.debta', 'D.L1.debta']].head(50)